# Stablecoin Billionaires<br> Descriptive Analysis of the Ethereum-based Stablecoin ecosystem 

## by Anton Wahrstätter, 01.07.2020

# Part I - Tether

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from collections import Counter
from matplotlib import rc
import re
import random

rc('font', **{'family':'serif','serif': ['Computer Modern']})
rc('text', usetex=True)

In [2]:
#plots
tx_over_date = '../plots/tether/tether_txs_over_date.csv'
unique_senders_over_date = '../plots/tether/tether_unique_senders_over_date.csv'
unique_recipients_over_date = '../plots/tether/tether_unique_recipients_over_date.csv'
tx_count_to = '../plots/tether/tether_tx_count_to.csv'
tx_count_from = '../plots/tether/tether_tx_count_from.csv'
tx_over_date = '../plots/tether/tether_txs_over_date.csv'
balances = '../plots/tether/tether_balances.csv'
avg_gas_over_date = '../plots/tether/tether_avg_gas_over_date.csv'
avg_value_over_date = '../plots/tether/tether_avg_value_over_date.csv'
positive_cumulated_balances = '../plots/tether/tether_positive_cumulated_balances.csv'
circulating_supply = '../plots/tether/tether_circulating_supply.csv'
unique_recipients_per_day_over_date = '../plots/tether/tether_unique_recipients_per_day_over_date.csv'
unique_senders_per_day_over_date = '../plots/tether/tether_unique_senders_per_day_over_date.csv'
exchanges = '../plots/exchanges.csv'

#data
chunk0 = '../data/tether/transfer/0_tether_transfer_4638568-8513778.csv'
chunk1 = '../data/tether/transfer/1_tether_transfer_8513799-8999999.csv'
chunk2 = '../data/tether/transfer/2_tether_transfer_9000000-9799999.csv'
chunk3 = '../data/tether/transfer/3_tether_transfer_9800000-10037842.csv'
chunk4 = '../data/tether/transfer/4_tether_transfer_10037843-10176690.csv'
chunk5 = '../data/tether/transfer/5_tether_transfer_10176691-10370273.csv' 
transfer = '../data/tether/transfer/tether_transfers.csv'
issue = '../data/tether/issue/tether_issue.csv'
destroyedblackfunds = '../data/tether/destroyedblackfunds/tether_destroyedblackfunds.csv'

<center></center>

# Data

In [3]:
df = pd.read_csv(transfer)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

## Basics

In [4]:
df

,timestamp,blocknumber,txindex,txhash,txfrom,txto,txvalue,gas_price,gas_used
0,1511883490,4638568,24,0x51a2395087450379f38a866662090d6656e73929d205...,0x36928500bc1dcd7af6a2b4008875cc336b927d57,0xc6cde7c39eb2f0f0095f41570af89efc2c1ea828,100000000,4000000000,53401
1,1511883650,4638577,133,0x2c22ad3e367ee78da808ffec1e03a677ebb30703ce6c...,0xc6cde7c39eb2f0f0095f41570af89efc2c1ea828,0x2b2c9153acc4021676c165dc3e3b1c7ad1c7849d,10000000,10100000000,78185
2,1511883825,4638586,40,0x4afc4e7f282e9de371199df62803ce1f51722f1c669f...,0x36928500bc1dcd7af6a2b4008875cc336b927d57,0xc6cde7c39eb2f0f0095f41570af89efc2c1ea828,99900000000,4000000000,23465
3,1511885010,4638663,14,0x35348a160fd8b86adea49ff482f0d9d0185d5444206f...,0xc6cde7c39eb2f0f0095f41570af89efc2c1ea828,0x2b2c9153acc4021676c165dc3e3b1c7ad1c7849d,10000000,21000000000,208890
4,1511885455,4638700,18,0xe544a6ac188ea9c71427907974b90351c6b9ca2555c0...,0x2b2c9153acc4021676c165dc3e3b1c7ad1c7849d,0x9faf5515f177f3a8a845d48c19032b33cc54c09c,20000000,27000000000,38401
...,...,...,...,...,...,...,...,...,...
39501955,1593561580,10370273,231,0x1099ba059b11305d0ba50182e2c691b193c32f3265fe...,0x32cf6b531611b23011f92b3949112ff504e9e905,0xf248e371b165bd8d7116b6aae59cd826aa100708,1412451625,36400000000,56209
39501956,1593561580,10370273,247,0x4f589a62725c3245ca792e3c5af01039bb784687a014...,0x9143e2176ee5db4157f79b944836755ffcd234c9,0xf0b97a87c3d03b7c139c5739f99db61ee554e600,129000000,33600000000,26197
39501957,1593561580,10370273,259,0x21631a7e2b788a68be78d325c896a666b19106c2c9e8...,0xa5407eae9ba41422680e2e00537571bcc53efbfd,0xdd222f0b646ec393160ae597037db51b5327cdde,500098611,31830000000,99675
39501958,1593561580,10370273,266,0x345ad323bbd4cf1e87f72632fb37bd65ff7f9478cf8c...,0x00a2b2a9d01a80d1c0219a4b4bc6a02541e3eaa3,0xd98ad997d1b974954782ebe7961336df7966538c,122636450,31000000000,41209


In [5]:
df.describe()

,timestamp,blocknumber,txindex,txvalue,gas_price,gas_used
count,39501960.000,39501960.000,39501960.000,39501960.000,39501960.000,39501960.000
mean,1580935679.401,9444054.995,72.188,8172894411.684,27244892832.678,57439.360
std,9584380.055,694142.685,57.457,292561831217.660,66752124163.847,226811.609
min,1511883490.000,4638568.000,0.000,0.000,0.000,22121.000
25%,1572446336.000,8840837.000,24.000,118180000.000,10500000002.000,26221.000
50%,1583306564.000,9603341.000,59.000,460000000.000,23352536853.500,41209.000
75%,1589548490.000,10070942.000,110.000,1827000000.000,37000000000.000,53465.000
max,1593561580.000,10370273.000,567.000,549037905787436.000,210010998142857.000,9430022.000


## Largest Transfer and Costs

In [6]:
#Largest tx: 0xf5b73c8fa7e8c0f0a06b8ecb809784143943c3b0abf665747eadf7b4610fa23e
la = pd.read_csv(chunk0)
la[la['txvalue']==max(la['txvalue'])]

,timestamp,blocknumber,txindex,txhash,txfrom,txto,txvalue,gas_price,gas_used
805347,1562684575,8117763,66,0xf5b73c8fa7e8c0f0a06b8ecb809784143943c3b0abf6...,0x4e9ce36e442e55ecd9025b9a6e0d88485d628a67,0xbe0eb53f46cd790cd13851d5eff43d12404d33e8,549037905787436,15000000000,38657


In [7]:
15000000000*38657/10**18

0.000579855

<center></center>

## Dataset

In [8]:
print('Start:')
print('Block: {:^30}\nTimestamp: {:^20}\nUTC Time: {:^25}\n'.format(df['blocknumber'].iloc[0],
                                                        df['timestamp'].iloc[0],
                                                        str(datetime.fromtimestamp(df['timestamp'].iloc[0]))
                                                       ))
print('End:')
print('Block: {:^30}\nTimestamp: {:^20}\nUTC Time: {:^25}\n'.format(df['blocknumber'].iloc[-1],
                                                        df['timestamp'].iloc[-1],
                                                        str(datetime.fromtimestamp(df['timestamp'].iloc[-1]))
                                                       ))

Start:
Block:            4638568            
Timestamp:      1511883490     
UTC Time:    2017-11-28 16:38:10   

End:
Block:            10370273           
Timestamp:      1593561580     
UTC Time:    2020-07-01 01:59:40   



## Total Nr. of Blocks

In [9]:
print('Total Nr. of Blocks: {}'.format(df['blocknumber'].iloc[-1]-df['blocknumber'].iloc[0]))

Total Nr. of Blocks: 5731705


<center></center>

## Total Nr. of Transfer Events

In [10]:
print('Total Nr. of Events: {:,.0f}'.format(df.describe().loc['count','timestamp']))

Total Nr. of Events: 39,501,960


<center></center>

## Total Nr. of Addresses

In [11]:
print('Total Nr. of Addresses: {}'.format(len(df['txto'].unique())))

Total Nr. of Addresses: 6299834


<center></center>

## Addresses with funds

In [12]:
bal = pd.read_csv(balances)
print('Total Nr. of Addresses with funds: {}'.format(len(bal[bal['txvalue']>0])))

Total Nr. of Addresses with funds: 1477063


<center></center>

## Avg. Transaction Value

In [13]:
print('Avg. Transaction Value: {:,.0f} tether'.format(np.mean(df['txvalue']/ 10**6)))

Avg. Transaction Value: 8,173 tether


<center></center>

## Total Gas Costs

In [14]:
df["costs"] = (df["gas_price"]/10**18) * df["gas_used"]
print('Total Gas spent for Transfers: {:,.3f}'.format(sum(df["costs"])))

Total Gas spent for Transfers: 60,146.923


<center></center>

## Initial Tether Supply

In [15]:
init = df[(df['txfrom'] == '0x36928500bc1dcd7af6a2b4008875cc336b927d57') & 
                     (df['timestamp'] < 1511883900)]['txvalue'].sum()
print('Initial Supply: {:,.0f} tether'.format(init/10**6))

Initial Supply: 100,000 tether


<center></center>

## Total Tether Supply

In [16]:
sum(pd.read_csv(issue)['txvalue']/10**6)-sum(pd.read_csv(destroyedblackfunds)['txvalue']/10**6)

6037847550.677356

<center></center>

## Balance Bitfinex Multisig as of July 1st

In [17]:
6040118005.642-6040009155.642


108850.0

<center></center>

<center></center>

# I. Event analysis

## I.I. Issue Event

## Plot new issued tokens over date

In [ ]:
print('\n\n')
fig = plt.figure(figsize=(40,25), dpi=250)
ax = fig.subplots()
plt.grid()
plt.title(r'I s s u e d \ \ T e t h e r'+'\n', size= 120)
ax.yaxis.get_offset_text().set_fontsize(50)
plt.xlabel('\n'+r'D a t e ', size=120)
plt.ylabel(r'U S D T'+'\n', size=120)
plt.yticks(fontsize=60)
plt.xticks(labels=["\nJan '18","Apr '18",
                   "\nJul '18","Oct '18","\nJan '19",
                   "Apr '19","\nJul '19","Oct '19",
                   "\nJan '20","Apr '20","\nJul '20"], 
           ticks=[34,124,
                  215,307,399,
                  489,580,672,
                  764,855,946], fontsize=60)

def plot_issue_over_date():
    _issue = pd.read_csv(issue)
    iss = _issue.loc[:, ['timestamp', 'txvalue']]
    iss['utc'] = iss['timestamp'].apply(lambda x: str(datetime.utcfromtimestamp(x))[0:10])
    iss = iss.groupby('utc', as_index = False)['txvalue'].sum()
    a = iss['utc'].iloc[0]
    b = iss['utc'].iloc[-1]
    idx = pd.date_range(a,b)
    iss = iss.set_index('utc')
    iss.index = pd.DatetimeIndex(iss.index)
    iss = iss.reindex(idx, fill_value=0)
    counter = 0
    for i in range(0, len(iss)):
        plt.plot([counter,counter], [0, iss['txvalue'].iloc[counter]/(10**6)], color= 'black', linewidth=5)
        counter += 1
    return 

plt.tight_layout(pad=5)
plot_issue_over_date()
plt.savefig('../pics/tether/tether_issued_tether_over_date.pdf')

## Further info

In [18]:
df = pd.read_csv(issue)

print('Issue Events: {}\nIssued Tether: {:,.0f}\n'.format(len(df), sum(df['txvalue'])/10**6, ":,0f"))
print('Largest issue: {:,.0f} Tether\n . . . to address: {}\n'.format(df.loc[72, 'txvalue']//10**6,'0xC6CDE7C39eB2f0F0095F41570af89eFC2C1Ea828'))

Issue Events: 74
Issued Tether: 6,040,118,006

Largest issue: 120,000,000 Tether
 . . . to address: 0xC6CDE7C39eB2f0F0095F41570af89eFC2C1Ea828



<center></center>

<center></center>

## I.II. DestroyBlackFunds Event

## Plot burned tokens over date

In [ ]:
print('\n\n')
fig = plt.figure(figsize=(40,25))
ax = fig.subplots()
plt.grid()
plt.title(r'B u r n e d \ \ T e t h e r'+'\n', size= 120)
ax.yaxis.get_offset_text().set_fontsize(50)
plt.xlabel('\n'+r'D a t e', size=120)
plt.ylabel(r'U S D T'+'\n', size=120)
plt.yticks(fontsize=60)
plt.xticks(labels=["\nJan '18","Apr '18",
                   "\nJul '18","Oct '18","\nJan '19",
                   "Apr '19","\nJul '19","Oct '19",
                   "\nJan '20","Apr '20","\nJul '20"], 
           ticks=[34,124,
                  215,307,399,
                  489,580,672,
                  764,855,946], fontsize=60) 

def plot_burn_over_date():
    _dbf = pd.read_csv(destroyedblackfunds)
    dbf = _dbf.loc[:, ['timestamp', 'txvalue']]
    dbf['utc'] = dbf['timestamp'].apply(lambda x: str(datetime.utcfromtimestamp(x))[0:10])
    dbf = dbf.groupby('utc', as_index = False)['txvalue'].sum()
    a = dbf['utc'].iloc[0]
    b = dbf['utc'].iloc[-1]
    idx = pd.date_range(a,b)
    dbf = dbf.set_index('utc')
    dbf.index = pd.DatetimeIndex(dbf.index)
    dbf = dbf.reindex(idx, fill_value=0)
    counter = 0
    for i in range(0, len(dbf)):
        plt.plot([counter,counter], [0, dbf['txvalue'].iloc[counter]/(10**6)], color= 'black', linewidth=5)
        counter += 1
    return

plt.tight_layout(pad=5)
plot_burn_over_date()
plt.savefig('../pics/tether/tether_burned_tether_over_date.pdf')

## Further info

In [19]:
df = pd.read_csv(destroyedblackfunds)

print('Burn Events: {}\nBurned Tether: {:,.0f}'.format(len(df), sum(df['txvalue'])/10**6, ":,0f"))
print('. . . from {} addesses\n'.format(len(df['address'].unique())))
print('Largest burn: {:,.0f} Tether\n . . . from address: {}\n'.format(df.groupby("address")["txvalue"].sum()[0]/10**6,df.groupby("address")["txvalue"].sum().index[0]))

Burn Events: 49
Burned Tether: 2,270,455
. . . from 32 addesses

Largest burn: 959,534 Tether
 . . . from address: 0x0000000000000000000000000000000000000000



<center></center>

<center></center>

## Plot circulating supply

In [ ]:
pd.read_csv(circulating_supply, index_col='Unnamed: 0').cumsum()

In [ ]:
print('\n\n')
fig = plt.figure(figsize=(20,12), dpi=500)
ax = fig.subplots()
plt.grid(True)
plt.title(r'C i r c u l a t i n g \ \ T e t h e r \ \ S u p p l y'+'\n', size=60)
plt.xlabel('\n'+r'D a t e', size= 60)
plt.ylabel(r'U S D T'+'\n', size= 60)
ax.yaxis.get_offset_text().set_fontsize(25)

plt.yticks(fontsize=30)
plt.xticks(labels=["\nJan '18","Apr '18",
                       "\nJul '18","Oct '18","\nJan '19",
                       "Apr '19","\nJul '19","Oct '19",
                       "\nJan '20","Apr '20","\nJul '20"], 
               ticks=[34,124,
                      215,307,399,
                      489,580,672,
                      764,855,946], fontsize=30)

circ = pd.read_csv(circulating_supply, index_col='Unnamed: 0')
plt.plot(range(0, 945), circ['txvalue'].cumsum()/10**6, color='black', linewidth = 4, label = 'Tether supply')
plt.fill_between(range(0, 945),0 , circ['txvalue'].cumsum()/10**6, alpha=0.2, facecolor='#2D728F')
lgnd = plt.legend(loc='upper left', fontsize=40)
plt.tight_layout(pad=5)
plt.savefig('../pics/tether/tether_cirulating_supply.pdf')

<center></center>

<center></center>

<center></center>

## I.III. Transfer Event

## Plot transfers over date

In [ ]:
print('\n\n')
fig = plt.figure(figsize=(20,12), dpi=500)
ax = fig.subplots()
plt.grid(True)
plt.title(r'T e t h e r \ \ T r a n s f e r s'+'\n', size=60)
plt.xlabel('\n'+r'D a t e', size= 50)
plt.ylabel(r'T r a n s f e r s'+'\n', size= 50)
plt.yticks(np.arange(0, 300001, 50000), 
           np.vectorize(lambda x: f'{x:,.0f}')(np.arange(0, 300001, 50000)), 
           fontsize=30)
plt.xticks(labels=["\nJan '18","Apr '18",
                       "\nJul '18","Oct '18","\nJan '19",
                       "Apr '19","\nJul '19","Oct '19",
                       "\nJan '20","Apr '20","\nJul '20"], 
               ticks=[34,124,
                      215,307,399,
                      489,580,672,
                      764,855,946], fontsize=30)

def plot_txs_over_date(df, lwd, label, col = '#2D728F', fillbetween=False):
    plt.plot(np.arange(0 , len(df["txs"])), df["txs"], color = col, linewidth = lwd, label = label)
    if fillbetween:
        plt.fill_between(np.arange(0 , len(df['txs'])),0 , df['txs'], alpha=0.2, facecolor='#2D728F')
 
    
plot_txs_over_date(df = pd.read_csv(tx_over_date, index_col=0), 
                   col = 'black', lwd = 2, label = 'Transfers', fillbetween=True)

plot_txs_over_date(pd.read_csv(unique_recipients_per_day_over_date, index_col='Unnamed: 0'), 
                   lwd = 2, label = 'Unique Recipients per day')

plot_txs_over_date(pd.read_csv(unique_senders_per_day_over_date, index_col='Unnamed: 0'), 
                   col='#9DB469', lwd = 2, label =  'Unique Senders per day')

lgnd = ax.legend(loc='upper left', fontsize=35)
lgnd.legendHandles[0].set_linewidth(5.0)
lgnd.legendHandles[1].set_linewidth(5.0)
lgnd.legendHandles[2].set_linewidth(5.0)

plt.tight_layout(pad=5)
plt.savefig('../pics/tether/tether_tx_over_date.pdf')
plt.show()

<center></center>

<center></center>

## Plot unique transfers over date

In [ ]:
print('\n\n')
fig = plt.figure(figsize=(20,12), dpi=500)
ax = fig.subplots()
plt.grid(True)
plt.title(r'T r a n s f e r s \ \ o v e r \ \ D a t e'+'\n', size=60)
plt.xlabel('\n'+r'D a t e', size= 50)
plt.ylabel(r'T r a n s f e r s'+'\n', size= 50)
plt.yticks(np.arange(0, 60000, 10000), 
           np.vectorize(lambda x: f'{x:,.0f}')(np.arange(0, 60000, 10000)), 
           fontsize=30)
plt.xticks(labels=["\nJan '18","Apr '18",
                       "\nJul '18","Oct '18","\nJan '19",
                       "Apr '19","\nJul '19","Oct '19",
                       "\nJan '20","Apr '20","\nJul '20"], 
               ticks=[34,124,
                      215,307,399,
                      489,580,672,
                      764,855,946], fontsize=30)

def plot_txs_over_date(df, lwd, label, col = '#2D728F'):
    plt.plot(np.arange(0 , len(df['txs'])), df['txs'], color = col, linewidth = lwd, label = label)
      

plot_txs_over_date(df = pd.read_csv(unique_senders_over_date, index_col=0), 
                   col = '#87AEDF', lwd = 2, label = 'Unique Senders')

plot_txs_over_date(df = pd.read_csv(unique_recipients_over_date, index_col=0), 
                   col = 'black', lwd = 2, label = 'Unique Recipients')



lgnd = ax.legend(loc='upper left', fontsize=35)
lgnd.legendHandles[0].set_linewidth(5.0)
lgnd.legendHandles[1].set_linewidth(5.0)

plt.tight_layout(pad=5)
plt.savefig('../pics/tether/tether_unique_tx_over_date.pdf')
plt.show()

<center></center>

<center></center>

## Most active addresses

From:

In [20]:
fr = pd.read_csv(tx_count_from, index_col='Unnamed: 0').sort_values('txs', ascending = False)
to = pd.read_csv(tx_count_to, index_col='Unnamed: 0').sort_values('txs', ascending = False)
fr = pd.DataFrame(fr.loc[:fr.index[10],'txs'])
fr['tag'] = ['Huobi 2', 'Huobi 1', 'Huobi 9', 'Huobi 3', 'Huobi 12', 'Huobi 7', 'Huobi 4', 'Huobi', 'Huobi 10', 'Binance', 'OKEx 1']
fr

,txs,tag
0x6748f50f686bfbca6fe8ad62b22228b87f31ff2b,816526,Huobi 2
0xab5c66752a9e8167967685f1450532fb96d5d24f,814541,Huobi 1
0x1062a747393198f70f71ec65a582423dba7e5ab3,789140,Huobi 9
0xfdb16996831753d5331ff813c29a93c76834a0ad,710860,Huobi 3
0x46705dfff24256421a05d056c29e81bdc09723b8,688401,Huobi 12
0xadb2b42f6bd96f5c65920b9ac88619dce4166f94,677574,Huobi 7
0xeee28d484628d41a82d01e21d12e2e78d69920da,669777,Huobi 4
0x0a98fb70939162725ae66e626fe4b52cff62c2e5,668977,Huobi
0xe93381fb4c4f14bda253907b18fad305d799241a,655901,Huobi 10
0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,456880,Binance


To:

In [21]:
to = pd.DataFrame(to.loc[:to.index[10],'txs'])
to['tag'] = ['Binance', 'OKEx 1', 'Huobi 2', 'Huobi 1', 'Huobi 9', 'Huobi', 'Huobi', 'Huobi 4', 'Huobi 3', 'Huobi 7', 'Huobi']
to

,txs,tag
0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,653580,Binance
0x6cc5f688a315f3dc28a7781717a9a798a59fda7b,318243,OKEx 1
0x6748f50f686bfbca6fe8ad62b22228b87f31ff2b,277820,Huobi 2
0xab5c66752a9e8167967685f1450532fb96d5d24f,274992,Huobi 1
0x1062a747393198f70f71ec65a582423dba7e5ab3,271727,Huobi 9
0xc6bd3edd07e294cb66b8318356d688b3516ea950,267313,Huobi
0x4c3dc6d5d3fc47060b5644d057aeb12d31fa79ee,248475,Huobi
0xeee28d484628d41a82d01e21d12e2e78d69920da,244776,Huobi 4
0xfdb16996831753d5331ff813c29a93c76834a0ad,244752,Huobi 3
0xadb2b42f6bd96f5c65920b9ac88619dce4166f94,244674,Huobi 7


<center></center>

<center></center>

## Activity distribution

In [22]:
df_from = pd.read_csv(tx_count_from, index_col=0)
df_to = pd.read_csv(tx_count_to, index_col=0)
df_all = pd.concat([df_from, df_to])
df = df_all.groupby(df_all.index).sum()
print('{} addresses in total'.format(len(df)))
df = df.sort_values('txs')
gr0 = len(df.loc[df['txs'] >= 500000])
gra = len(df.loc[df['txs'] >= 100000]) - gr0 
grb = len(df.loc[df['txs'] >= 50000]) - gr0 - gra
grc = len(df.loc[df['txs'] >= 10000]) - gr0 - gra - grb
grd = len(df.loc[df['txs'] >= 1000]) - gr0 - gra - grb - grc
gre = len(df.loc[df['txs'] >= 100]) - gr0 - gra - grb - grc - grd
grf = len(df.loc[df['txs'] >= 10]) - gr0 - gra - grb - grc - grd - gre
grg = len(df.loc[df['txs'] <= 10])
grh = len(df.loc[df['txs'] == 1]) 
pd.DataFrame({'Transactions': ['> 500.000','100.000-500.000',
                               '50.000-100.000',
                               '10.000-50.000',
                               '1.000-10.000',
                               '100-1.000',
                               '10-100',
                               '< 10', 
                               '1'], 
              'Addresses':[gr0,gra,grb,grc,grd,gre,grf,grg,grh]
             })

6301972 addresses in total


,Transactions,Addresses
0,> 500.000,12
1,100.000-500.000,29
2,50.000-100.000,38
3,10.000-50.000,321
4,1.000-10.000,3106
5,100-1.000,50568
6,10-100,708268
7,< 10,5626141
8,1,514435


<center></center>

<center></center>

## Transaction != Transfer

In [23]:
df = pd.read_csv(chunk5)

In [24]:
_df = dict(Counter(df[df['txhash'].duplicated()]['txhash'].tolist()))

In [25]:
has = max(_df, key=_df.get)

In [27]:
max(_df.values())

250

In [28]:
len(df[df['txhash']=='0xe49af17de711d7ca1f034f2fb821082a83a50b53d8b4a649addbc0b87d0c3faf']['txto'][df[df['txhash']=='0xe49af17de711d7ca1f034f2fb821082a83a50b53d8b4a649addbc0b87d0c3faf']['txto'].duplicated()])

19

In [29]:
has

'0xd4a015df7729a7dac116b5d00f2ff0387d5564a4be8b66cd6dbb3ed0e891ba7d'

<center></center>

<center></center>

# Transfer Amount

## Jan '20 - Jul '20

In [30]:
df = pd.read_csv(avg_value_over_date, index_col=0)
df = df.loc[df.index[764]:,:]

In [ ]:
print('\n\n')
plt.figure(figsize=(12, 7), dpi=800)
plt.grid(True)
plt.plot(np.arange(0 , len(df.index.tolist())), df['txvalue'], color = 'black', label = 'Avg. Amount/Day', linewidth = 2)
plt.fill_between(np.arange(0 , len(df.index.tolist())),0 , df['txvalue'], alpha=0.2, facecolor='#2D728F')
plt.xlabel('\n'+'D a t e', fontsize=35)
plt.ylabel('U S D T'+'\n', fontsize=30)
plt.title("T e t h e r\nA v g. \ \ T r a n s f e r \ \ A m o u n t"+"\n", size = 30)
plt.legend(loc="upper right", fontsize=20, shadow= True)
plt.ticklabel_format(style = 'plain')
plt.xticks(labels=["\nJan '20","Feb '20","\nMar '20","Apr '20","\nMay '20","Jun '20","\nJul '20"], 
           ticks=[0,31,60,90,121,152,182], fontsize=23) 
plt.yticks(np.arange(0, 50001, 10000), 
           np.vectorize(lambda x: f'{x:,.0f}')(np.arange(0, 50001, 10000)), 
           fontsize=15)
plt.tight_layout(pad=1)
plt.savefig('../pics/tether/tether_avgtxvalue_jan20.pdf')

<center></center>

## Further Info

In [31]:
df.describe()

,txvalue
count,182.000
mean,8134.001
std,2603.998
min,2668.441
25%,6393.337
50%,8009.014
75%,9495.828
max,20665.404


<center></center>

<center></center>

<center></center>

In [32]:
#some expensive ones
aa = pd.read_csv(chunk5)
aa["costs"] = (aa["gas_price"]/10**18).multiply(aa["gas_used"])
aa.sort_values("costs")

,timestamp,blocknumber,txindex,txhash,txfrom,txto,txvalue,gas_price,gas_used,costs
6177356,1593313610,10351725,166,0x5c89c1eaf05b5ff467aa8c7cc764804d7ff4690fac94...,0x02ed3d40f0cf56ed5e5a827e69879a37acea109d,0xfdea36043bb04c1184330a985efa2bf0e0d9aa79,10990753,540000000,41197,0.000
943524,1591342044,10204388,175,0x83bc924caf3b3c3ea64bcca949c1db6479e1db978224...,0xd7ef77a93cd5db35589a58fc0e9aa7971ee96170,0x369aacc1fc5e691eaca1f342c4ebb046c5117573,1,1000000000,41173,0.000
2538661,1591933909,10248556,164,0x915165d3c7734f93dd672e628bf3c10abdc87d627725...,0x3f0e0c18efa50f91c0f0f30ccbd74b0feabb47d7,0xf80e216ab8aa72be5432ca75b1a9afdc2225d588,4000000,1000000000,41185,0.000
2833351,1592044102,10256813,136,0x8b1d3fd076bd8e9cd015db91546988931ded9645ac7b...,0x88d675f2420a7f17eeb510654b080a401bcc21cd,0xccf28f4f15b1aafd1f84bf037610845c592aebe7,13713342,1000000000,41197,0.000
3064859,1592166199,10265987,141,0x7553520d9f0eeda6997af46cc50c89e8c6ca8f2bed56...,0xb90432f943ac620338067ba44bb2ddb98b6445c2,0xbc7e71db5cbf25255ea69f298f8affd54234a558,10000000,1000000000,41197,0.000
...,...,...,...,...,...,...,...,...,...,...
708915,1591247557,10197363,0,0x0f5304457800bc922fe2e63c8237672f50ce53fdaca9...,0x72e6e05a3459e1856d579c70ff6cf6e48f0e99cb,0x962e88bc7ae942c080a7a8d9e5d83d2585b8859c,354000000,181261353781428,26209,4.751
1264360,1591457926,10213001,0,0xd704e6848868e0a75b3718a8f66eee089ac9046a4bfb...,0xdd041298c3b126aa198a1a1fe6718ba9c5bd1478,0x962e88bc7ae942c080a7a8d9e5d83d2585b8859c,30300000,210010998142857,26209,5.504
5631881,1593087635,10334888,0,0xafae82ebac25289b0384cfd1554cb27b00dad4f81c8c...,0x8c863333c2e92f02e01f7a3c6d131e4d59f78990,0xf650c3d88d12db855b8bf7d11be6c55a4e07dcc9,93194794804,4244493944360,1507681,6.399
5631880,1593087635,10334888,0,0xafae82ebac25289b0384cfd1554cb27b00dad4f81c8c...,0xf650c3d88d12db855b8bf7d11be6c55a4e07dcc9,0x8c863333c2e92f02e01f7a3c6d131e4d59f78990,93192206235,4244493944360,1507681,6.399


# Gas Costs

<center></center>

## Jan '20 - Jul '20

In [33]:
df = pd.read_csv(avg_gas_over_date)
df = df.loc[df.index[764]:,:]

In [ ]:
print('\n\n')
plt.figure(figsize=(12, 7), dpi=800)
plt.grid(True)
plt.plot(np.arange(0 , len(df.index.tolist())), df['gas'], color = 'black', label = 'Avg. Gas Costs/Day', linewidth = 2)
plt.fill_between(np.arange(0 , len(df.index.tolist())),0 , df['gas'], alpha=0.2, facecolor='#2D728F')
plt.xlabel('\nD a t e', fontsize=35)
plt.ylabel('E t h e r\n', fontsize=30)
plt.title('T e t h e r\nA v g. \ \ G a s \ \ C o s t s\n', size = 30)
lgnd = plt.legend(loc='upper left', fontsize=20, shadow= True)
plt.ticklabel_format(style = 'plain')
plt.xticks(labels=["\nJan '20","Feb '20","\nMar '20","Apr '20","\nMay '20","Jun '20","\nJul '20"], 
           ticks=[0,31,60,90,121,152,182], fontsize=20) 
plt.yticks(np.arange(0, 0.05, 0.01), 
           np.vectorize(lambda x: f'{x:,.3f}')(np.arange(0, 0.05, 0.01)), 
           fontsize=15)
plt.tight_layout(pad=1)

lgnd.legendHandles[0].set_linewidth(3.0)
plt.savefig('../pics/tether/tether_avggascosts_jan20.pdf') 

## Further Info 

In [34]:
pd.set_option('display.float_format', lambda x: '%.6f' % x)
df.describe()

,gas
count,182.000000
mean,0.001578
std,0.001083
min,0.000680
25%,0.000879
50%,0.001123
75%,0.002146
max,0.010481


<center></center>

<center></center>

# II. Balances Analysis

In [35]:
df = pd.read_csv(positive_cumulated_balances, index_col="Unnamed: 0")
df

/home/nero/python/master_thesis/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,address,balance,cum
0,0x3527995dc66caef918fd97855854e4afe2faa869,0.000001,0.000001
1,0xd559f7518a7b4208c32fe76aff704e917b61c927,0.000001,0.000002
2,0xb55a57255b387dfeaf51d08f81eadf42007e00ec,0.000001,0.000003
3,0xdac4d16e24ccc324b762109467ea9b9d3b231d41,0.000001,0.000004
4,0x801bd853627dc527c95c8337b92caaef8720555c,0.000001,0.000005
...,...,...,...
1477057,0x5754284f345afc66a98fbb0a0afe71e0f007b949,139291708.339840,4985660847.421669
1477058,0xfdb16996831753d5331ff813c29a93c76834a0ad,152457482.812605,5138118330.234274
1477059,0x6748f50f686bfbca6fe8ad62b22228b87f31ff2b,198610436.106398,5336728766.340672
1477060,0xbe0eb53f46cd790cd13851d5eff43d12404d33e8,301118650.039683,5637847416.380355


<center></center>

## II.I. Quick Summary

In [36]:
(df[df['balance']>0]['balance']/1000000).describe().apply(lambda x: format(x, 'f'))

count    1477062.000000
mean           0.004088
std            0.511874
min            0.000000
25%            0.000000
50%            0.000010
75%            0.000075
max          400.000001
Name: balance, dtype: object

<center></center>

## II.II. Balance Table

In [37]:
df = pd.read_csv(positive_cumulated_balances, index_col=0)
def get_distribution(perc):
    per = round(df.index[-1]*perc)
    entities = df.index[-1]- per
    upper = df.loc[per:,:]
    lower = df.loc[:per,:]
    lower_ = lower['cum'].iloc[-1] /(10*10**5)
    upper_ = (upper['cum'].iloc[-1] - upper['cum'].iloc[0])/(10*10**5)
    return entities, lower_, upper_, lower_/ upper['cum'].iloc[-1]/(10*10**5), upper_/(upper['cum'].iloc[-1]/(10*10**5))

idx90, lower90, upper90, per10, per90 = get_distribution(0.90)
idx95, lower95, upper95, per05, per95 = get_distribution(0.95)
idx99, lower99, upper99, per01, per99 = get_distribution(0.99)
idx999, lower999, upper999, per001, per999 = get_distribution(0.999)

df = pd.DataFrame([[f'{idx999:,.0f}', round(per999*100,2), f'{upper999:,.0f}'],
             [f'{idx99:,.0f}', round(per99*100,2),f'{upper99:,.0f}'],
             [f'{idx95:,.0f}', round(per95*100,2),f'{upper95:,.0f}'],
             [f'{idx90:,.0f}', round(per90*100,2),f'{upper90:,.0f}']],
                  index=['0.1% of the richest accounts', '1% of the richest accounts','5% of the richest accounts','10% of the richest accounts'],
                   columns=['Accounts in total', '% of total supply', 'Tether amount'])
df

/home/nero/python/master_thesis/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,Accounts in total,% of total supply,Tether amount
0.1% of the richest accounts,"1,477",80.350000,"4,851"
1% of the richest accounts,"14,771",96.000000,"5,796"
5% of the richest accounts,"73,853",98.850000,"5,968"
10% of the richest accounts,"147,706",99.350000,"5,999"


<center></center>

<center></center>

## II.III. Rich list

In [38]:
pd.options.mode.chained_assignment = None
df = pd.read_csv(positive_cumulated_balances)
balance = df
rich = df.loc[df.index[-10]:,:]
ex = pd.read_csv(exchanges, header=None)
loop = rich.iterrows()
for i, j in loop:
    if j['address'] in ex[0].tolist():
        rich.loc[i,'nametag'] = ex[ex[0] == j['address']][1].values[0]
print('0x7b8c69a0f660cd43ef67948976daae77bc6a019b: all tether transfers from Huobi 2 ')
print('0x5754284f345afc66a98fbb0a0afe71e0f007b949: Bitfinex')
print('0x5041ed759dd4afc3a72b8192c143f72f4724081a: OKEx')

rich.loc[1477085,'nametag'] = 'Huobi'
rich.loc[1477081,'nametag'] = 'Bitfinex'
rich.loc[1477079,'nametag'] = 'OKEx'
rich

0x7b8c69a0f660cd43ef67948976daae77bc6a019b: all tether transfers from Huobi 2 
0x5754284f345afc66a98fbb0a0afe71e0f007b949: Bitfinex
0x5041ed759dd4afc3a72b8192c143f72f4724081a: OKEx


,Unnamed: 0,address,balance,cum,nametag
1477052,1477052.000000,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,48898196.515196,4585938678.788507,Binance
1477053,1477053.000000,0xf650c3d88d12db855b8bf7d11be6c55a4e07dcc9,50338228.375235,4636276907.163742,Compound Tether
1477054,1477054.000000,0x66f820a414680b5bcda5eeca5dea238543f42054,52000000.000000,4688276907.163742,Bittrex 3
1477055,1477055.000000,0x5041ed759dd4afc3a72b8192c143f72f4724081a,58992008.040176,4747268915.203918,OKEx
1477056,1477056.000000,0x742d35cc6634c0532925a3b844bc454e4438f44e,99100223.877911,4846369139.081829,Bitfinex 2
1477057,1477057.000000,0x5754284f345afc66a98fbb0a0afe71e0f007b949,139291708.339840,4985660847.421669,Bitfinex
1477058,1477058.000000,0xfdb16996831753d5331ff813c29a93c76834a0ad,152457482.812605,5138118330.234274,Huobi 3
1477059,1477059.000000,0x6748f50f686bfbca6fe8ad62b22228b87f31ff2b,198610436.106398,5336728766.340672,Huobi 2
1477060,1477060.000000,0xbe0eb53f46cd790cd13851d5eff43d12404d33e8,301118650.039683,5637847416.380355,Binance 7
1477061,1477061.000000,0x7b8c69a0f660cd43ef67948976daae77bc6a019b,400000001.030000,6037847417.410355,Huobi


<center></center>

<center></center>

## Huobi

In [39]:
ex = pd.read_csv(exchanges, header=None)

In [40]:
df = ex.loc[0:73,:]
bal = 0 
for i in df[0]:
    val = balance['balance'][balance['address'] == i]
    if not val.empty:
        bal += balance['balance'][balance['address'] == i].values[0]
    else:
        pass
print('Huobi Total Balance: {:.0f}\n{:.2f}% of Total'.format(bal/(10**6), bal/balance.loc[balance.index[-1], 'cum']*100))

Huobi Total Balance: 764
12.66% of Total


<center></center>

## Binance

In [41]:
df = ex.loc[74:88,:]
bal = 0 
for i in df[0]:
    val = balance['balance'][balance['address'] == i]
    if not val.empty:
        bal += balance['balance'][balance['address'] == i].values[0]
    else:
        pass
print('Binance Total Balance: {:.0f}\n{:.2f}% of Total'.format(bal/(10**6), bal/balance.loc[balance.index[-1], 'cum']*100))

Binance Total Balance: 379
6.28% of Total


<center></center>

## Bitfinex

In [42]:
df = ex.loc[89:110,:]
bal = 0 
for i in df[0]:
    val = balance['balance'][balance['address'] == i]
    if not val.empty:
        bal += balance['balance'][balance['address'] == i].values[0]
    else:
        pass
print('Bitfinex Total Balance: {:.0f}\n{:.2f}% of Total'.format(bal/(10**6), bal/balance.loc[balance.index[-1], 'cum']*100))

Bitfinex Total Balance: 287
4.75% of Total


<center></center>

## OKEx

In [43]:
df = ex.loc[111:115,:]
bal = 0 
for i in df[0]:
    val = balance['balance'][balance['address'] == i]
    if not val.empty:
        bal += balance['balance'][balance['address'] == i].values[0]
    else:
        pass
print('OKEx Total Balance: {:.0f}\n{:.2f}% of Total'.format(bal/(10**6), bal/balance.loc[balance.index[-1], 'cum']*100))

OKEx Total Balance: 59
0.98% of Total


<center></center>

## Bittrex

In [44]:
df = ex.loc[116:119,:]
bal = 0 
for i in df[0]:
    val = balance['balance'][balance['address'] == i]
    if not val.empty:
        bal += balance['balance'][balance['address'] == i].values[0]
    else:
        pass
print('Bittrex Total Balance: {:.0f}\n{:.2f}% of Total'.format(bal/(10**6), bal/balance.loc[balance.index[-1], 'cum']*100))

Bittrex Total Balance: 56
0.92% of Total


<center></center>

## Compound

In [45]:
df = ex.loc[151:179,:]
bal = 0 
for i in df[0]:
    val = balance['balance'][balance['address'] == i]
    if not val.empty:
        bal += balance['balance'][balance['address'] == i].values[0]
    else:
        pass
print('Compound Total Balance: {:.0f}\n{:.2f}% of Total'.format(bal/(10**6), bal/balance.loc[balance.index[-1], 'cum']*100))

Compound Total Balance: 50
0.83% of Total


<center></center>

## Poloniex

In [46]:
df = ex.loc[247:266,:]
bal = 0 
for i in df[0]:
    val = balance['balance'][balance['address'] == i]
    if not val.empty:
        bal += balance['balance'][balance['address'] == i].values[0]
    else:
        pass
print('Poloniex Total Balance: {:.0f}\n{:.2f}% of Total'.format(bal/(10**6), bal/balance.loc[balance.index[-1], 'cum']*100))

Poloniex Total Balance: 22
0.36% of Total


<center></center>

<center></center>

<center></center>

# II. IV. Pie Chart

In [ ]:
df = pd.read_csv(positive_cumulated_balances, index_col="Unnamed: 0")
aa = df.iloc[df.index[-1]-80:]
bb = df['balance'].iloc[:df.index[-1]-80]
df = aa.append(pd.DataFrame({'address': 'others', 'balance': sum(bb)}, index=[0]))
label = []
counter = 0
def getlabel(i):
    global counter
    if i:
        if not i == 'others':
            label.append(i + '...')
        else:
            label.append(i)
    else:
        label.append('')
    counter += 1
    
[getlabel(i[:6]) if counter >= len(df)-11 else getlabel('') for i in df['address']  ]
print()

In [ ]:
print('\n\n')
# Colorspace colors by: https://colorspace.r-forge.r-project.org/index.html

colorspace_set3 = ['#EEBD92','#FFB3B5','#85D0F2','#BCC3FE','#E7B5F5', 
                 '#FEAFDA', '#61D8D6','#76D9B1','#A4D390','#CFC982']
colorsp_dynamic =['#DB9D85', '#87AEDF', '#9DB469', '#6DBC86', '#3DBEAB', 
                  '#4CB9CC', '#C2A968', '#BB9FE0', '#DA95CC', '#E494AB']
colorspa_dark_3 = ['#B675E0','#5991E4','#00AA5A','#6F9F00','#CE7D3B']
colorspa_dyna_5 = ['#9DB469','#87AEDF','#DA95CC', '#DB9D85','#3DBEAB']

fig = plt.figure(figsize=(25,15), dpi=400)
ax = fig.add_subplot()
aa = plt.pie(df['balance'],colors=colorsp_dynamic, 
             labels=label,
             autopct=lambda x: r'{:.1f}\%'.format(x) if x > 0.8 else r'{:.0f}\%'.format(x) if x > 1  else '', 
             pctdistance= 0.8, 
             labeldistance= 1.05, 
             radius=1, 
             explode=[0.05 for i in range(0, len(df['balance']))],
             wedgeprops = {'linewidth': 0.8, 'edgecolor':'k'}, 
             startangle=285) 

# Custom Modifications
aa[-1][-1].set_x(-0.7268917458682129)
aa[-1][-1].set_fontsize(35)
aa[-1][-2].set_fontsize(30)
aa[-1][-2].set_x(0.19977073082370535)
aa[-1][-2].set_y(0.8000006952023211)
aa[-1][-3].set_fontsize(27)
aa[-1][-4].set_fontsize(24)
aa[-1][-5].set_fontsize(20)
aa[-1][-6].set_fontsize(16)
aa[-1][-7].set_fontsize(13)
aa[-1][-8].set_fontsize(9)
aa[-1][-9].set_fontsize(9)
aa[-1][-10].set_fontsize(9)
aa[-1][-11].set_fontsize(8)

fontsize = -120
for i in aa[1]:
    i.set_fontsize(fontsize)
    fontsize += 2
aa[1][-1].set_fontsize(50)
plt.tight_layout(pad=5)
plt.title('T e t h e r \ \ D i s t r i b u t i o n', fontsize = 50)
circ = plt.Circle((0,0),0.5,color='black', fc='white',linewidth=1.25)
ax.add_artist(circ)
plt.savefig('../pics/tether/tether_distribution_pie.pdf')

## II.V. Lorenz curve

In [47]:
df = pd.read_csv(positive_cumulated_balances, index_col = "Unnamed: 0")
df

/home/nero/python/master_thesis/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,address,balance,cum
0,0x3527995dc66caef918fd97855854e4afe2faa869,0.000001,0.000001
1,0xd559f7518a7b4208c32fe76aff704e917b61c927,0.000001,0.000002
2,0xb55a57255b387dfeaf51d08f81eadf42007e00ec,0.000001,0.000003
3,0xdac4d16e24ccc324b762109467ea9b9d3b231d41,0.000001,0.000004
4,0x801bd853627dc527c95c8337b92caaef8720555c,0.000001,0.000005
...,...,...,...
1477057,0x5754284f345afc66a98fbb0a0afe71e0f007b949,139291708.339840,4985660847.421669
1477058,0xfdb16996831753d5331ff813c29a93c76834a0ad,152457482.812605,5138118330.234274
1477059,0x6748f50f686bfbca6fe8ad62b22228b87f31ff2b,198610436.106398,5336728766.340672
1477060,0xbe0eb53f46cd790cd13851d5eff43d12404d33e8,301118650.039683,5637847416.380355


In [48]:
y_all = df['cum']/df['cum'].iloc[-1]
x_all = (np.arange(start = 0 , stop = len(df['cum']), step = 1)/(len(df['cum'])))

y_25_75 = df['cum'].iloc[int(df.index[-1]*0.25):int(df.index[-1]*0.75)]
y_25_75 = y_25_75/max(y_25_75)
x_25_75 = np.arange(start = 0 , stop = len(y_25_75), step = 1)/(len(y_25_75))

In [49]:
print('Q3-Q1 (in ether):')
df['balance'].iloc[int(df.index[-1]*0.25):int(df.index[-1]*0.75)].describe().apply(lambda x: format(x/(10**6), 'f'))

Q3-Q1 (in ether):


count    0.738530
mean     0.000018
std      0.000021
min      0.000000
25%      0.000001
50%      0.000010
75%      0.000030
max      0.000075
Name: balance, dtype: object

In [ ]:
print('\n\n')
fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot()
plt.grid()
plt.title(r'L o r e n z \ \ C u r v e'+'\n', fontsize=50)
plt.xlabel('\n'+r'\% \ \ of \ \ A d d r e s s e s', fontsize=30)
plt.ylabel(r'\% \ o f \ \ t o t a l \ \ T e t h e r \ \ s u p p l y'+'\n',  fontsize=30)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
ax.plot(x_all,y_all, linewidth = 5, color = '#2D728F', label = r'$\ All$')
ax.plot(x_25_75,y_25_75, linewidth = 5, color = '#87AEDF', label = r'$\ Q_3 - Q_1$')
plt.legend(fontsize= 35)
plt.plot([0, 1], [0, 1], transform=ax.transAxes, linewidth = 4, ls = (0, (5, 10)), color = 'black')
ax.set_xlim([0,1.05])
plt.savefig('../pics/tether/tether_lorenzcurve.pdf')

<center></center>

# Graphs that didn't made it but are too nice to throw away

## Jan '19 - Jul '20

In [ ]:
df = pd.read_csv(avg_value_over_date, index_col=0)
df = df.loc[df.index[399]:,:]
plt.figure(figsize=(12, 7), dpi=800)

plt.grid(True)
plt.plot(np.arange(0 , len(df.index.tolist())), df['txvalue'], color = 'black', label = 'Avg. Amount/Day')
plt.xlabel('\n'+r'D a t e', fontsize=45)
plt.ylabel(r'U S D T'+'\n', fontsize=35)
plt.title(r"A v e r a g e \ \ T r a n s f e r \ \ A m o u n t"+"\n"+r"J a n \ \ ' 1 9 \ \ - \ \ J u l \ \ ' 2 0" + '\n', size = 30)
lgnd = plt.legend(loc="upper right", fontsize=20, shadow= True)
plt.ticklabel_format(style = 'plain')
plt.xticks(labels=["\nJan '19","Apr '19","\nJul '19","Oct '19",
                   "\nJan '20","Apr '20","\nJul '20"], 
           ticks=[0,90,181,273,
                  365,456, 547], fontsize=23) 
plt.yticks(np.arange(0, 400001, 50000), 
           np.vectorize(lambda x: f'{x:,.0f}')(np.arange(0, 400001, 50000)), 
           fontsize=15)
plt.tight_layout(pad=1)
lgnd.legendHandles[0].set_linewidth(3.0)
plt.savefig('../pics/tether/tether_avgtxvalue_jan19.pdf')

## Plot average gas costs

## Sep '17 - Jul '20

In [ ]:
print('\n\n')
df = pd.read_csv(avg_gas_over_date)
plt.figure(figsize=(12, 7), dpi=800)
plt.grid(True)
plt.plot(np.arange(0 , len(df.index.tolist())), df['gas'], color = 'black', label = 'Avg. Gas Costs/Day', linewidth = 1)
plt.xlabel('\n'+'D a t e', fontsize=45)
plt.ylabel('E t h e r'+'\n', fontsize=35)
plt.title("A v g. \ \ G a s \ \ C o s t s"+"\n"+"S e p \ \ ' 1 7 \ \ - \ \ J u l \ \ ' 2 0" + '\n', size = 30)
lgnd = plt.legend(loc="upper left", fontsize=20, shadow= True)
plt.ticklabel_format(style = 'plain')
plt.xticks(labels=["\nJan 18","Apr '18",
                       "\nJul '18","Oct '18","\nJan '19",
                       "Apr '19","\nJul '19","Oct '19",
                       "\nJan '20","Apr '20","\nJul '20"], 
               ticks=[34,124,
                      215,307,399,
                      489,580,672,
                      764,855,946], fontsize=20) 
plt.yticks(fontsize=15)
plt.tight_layout(pad=1)
lgnd.legendHandles[0].set_linewidth(3.0)
plt.savefig('../pics/tether/tether_avggascosts_sept17.pdf') 
    


## Jan '19 - Jul '20

In [ ]:
print('\n\n')
df = pd.read_csv(avg_gas_over_date)
df = df.iloc[399:,]
plt.figure(figsize=(12, 7), dpi=800)
plt.grid(True)
plt.plot(np.arange(0 , len(df.index.tolist())), df['gas'], color = 'black', label = 'Avg. Gas Costs/Day')
plt.xlabel('Date', fontsize=35)
plt.ylabel('Ether', fontsize=30)
plt.title("Average Gas Costs\nJan '19 - Jul '20\n", size = 30)
lgnd = plt.legend(loc="upper left", fontsize=20, shadow= True)
plt.ticklabel_format(style = 'plain')
plt.xticks(labels=["\nJan '19","Apr '19","\nJul '19","Oct '19",
                   "\nJan '20","Apr '20","\nJul '20"], 
           ticks=[0,90,181,273,
                  365,456, 547], fontsize=20) 
plt.yticks(fontsize=15)
plt.tight_layout(pad=1)
lgnd.legendHandles[0].set_linewidth(3.0)
plt.savefig('../pics/tether/tether_avggascosts_jan19.pdf') 

<center></center>